# Cleaning "Governments" Dataset

This notebook receives as input the raw edition of the Database of Political Institutions dataset from the Inter-American Development Bank. The notebook serves the purpose of addressing missing and inconsistent values, removing outliers, and creating consistent naming schemes to allow for cross-use with other datasets.

### About the Data
This dataset is provided by the Inter-American Development Bank (IDB). "The Database of Political Institutions presents institutional and electoral results data such as measures of checks and balances, tenure and stability of the government, identification of party affiliation and ideology, and fragmentation of opposition and government parties in the legislature ... [it covers] about 180 countries [from] 1975-2020. It has become one of the most cited databases in comparative political economy and comparative political institutions, with more than 4,500 article citations on Google Scholar as of December 2020." [(1)](https://publications.iadb.org/en/database-political-institutions-2020-dpi2020) For the context of this analysis, it includes 8,200 rows of data, each with 77 features. Throughout this analysis, this dataset will be referred to as the "Governments" dataset.


#### Data source citation:

Cruz, Cesi, Philip Keefer, and Carlos Scartascini. 2021. Database of Political Institutions 2020. Washington, DC: Inter-American Development Bank Research Department.

## Data Understanding

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Options
#pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
%matplotlib inline

# Convenience for working with external src code files
%load_ext autoreload
%autoreload 2

# Global constants used in data cleaning
NA_STRING = "NA_SS"
NA_NUMBER = -999.0
OTHER = "OTHER" 

##### Import Data

In [2]:
# Import data from stata dta file
path = '../data/raw/Database-of-Political-Institutions/DPI2020_stata13.dta'
countries = pd.read_stata(path)

# Use Numpy's NaN instead of Panda's for consistency
countries.fillna(np.nan, inplace=True) 

##### Simple cleaning

Note that the *remove_these_features* variable contains a list of features that were determined to be unvaluable based on extensive EDA. Given that there are 50+ features in that group, it is not worthwhile to explain each choice. Refer to the data dictionary to understand these specific features if desired.

In [3]:
# Convert type
countries['year'] = countries.year.dt.year 

# Drop unused features
remove_these_features = countries.iloc[:, 25:76].columns
countries.drop(remove_these_features, axis=1, inplace=True)

## Understanding columns

Below is a list of all features currently available for use, as well as their definitions.

- ifs - "IFS" country code
- system - Presidential (0), Assembly-elected president (1), parliamentry (2)
- yrsoffc - how many years has chief executive been in office?
- finittrm - is there a finite term? (0, 1)
- yrcurnt - years left in current term
- multpl - if there are formal restraints on an executive’s term (NA if not), can s/he serve additional term(s) following the current one?
- military - is chief exeuctive a military officer?
- defmin - is defense minister a military officer?
- PERCENT1 - president got what % of votes in the 1st/only round?
- PERCENTL - president got what % of votes in the final round? (na if no runoff)
- PRTYIN - party of chief executive has been how long in office
- EXECME - name of party, if any **remove** (too many distinct values)
- EXECRLC - executive is Right (1), Left (3), Center (2), No information (0), No executive (NA)
- EXECNAT - Nationalist (0, 1)
- EXECRURL - "Rural" issues listed as key component of party's platform? **remove** nearly no 1s
- EXECREG - "Regional" issues listed as key component of party's platform? **remove** nearly no 1s
- EXECREL - Executive religion
- EXECAGE - time since party formation (under same name)
- ALLHOUSE - does party of executive control all relevant houses?
- NONCHIEF - party affiliation of the one not called "Chief Executive" (in systems w/both non-ceremonial PM and president) **remove**
- TOTALSEATS - total seats in legislature. Includes gov1seat, gov2seat, gov3seat, opp1seat, opp2seat, opp3seat, govothst, oppthst, numul)
- GOV1ME, GOV1SEAT GOV1[ETC] - Descriptors of largest party. Too granular to bother with in this analysis since there are already aggregate features that account for the same data. **remove**
- OPPMAJH - does one opposition party have an absolute majority in House?
- OPPMAJS - does one opposition party have an absolute majority in Senate?
- DATELEG - month when parliamentary elections were held **remove**
- DATEEXEC - month when presidential elections were held **remove**
- LEGELEX - legislative election this year?
- EXELEC - executive election this year?
- LIEC, EIEC - Legislative, Executive Index of Electoral Completiveness (see data dictionary, valuable metric)
- MDMH, MDMS - Mean District Magnitude House, Senate
- PLURALTY - government is plurality?
- PR - proportional representation?
- HOUSESYS, SENSYS - House, Senate electorical rule
- THRESH - vote threshold for representation *in proportional representation system*
- DHONDT - is the D'Hondt system used? **remove**
- CL - are closed lists used? 
- GQ, GQI - gender quota, whether it was implemented
- SELECT - method for selecting election candidates **remove** (almost no values)

--------

- AUTON - are there autonomous regions? (federalism)
- MUNI - are municipal governments locally elected?
- STATE - are there state/providence governments locally elected? (important!)
- AUTHOR - do the state/provinves have authority over taxing, spending, or legislating? (important!)
- STCONST - are the constituencies of the senators the states/provides? **remove**
- GWNO - no idea, not included in dictioanry **remove**
- NUMGOV - total number of seats held by all government parties **remove**
- NUMVOTE - vote share of ruling government party  
- NUMOPP - vote share of opposition government party **remove**
- FRAUD - were vote fraud or candidate intimidation serious enough to affect the outcome of elections
- MAJ - Margin of Majority (important!)
- PARTYAGE - average age of parties
- HERFGOV, HERFOPP - Herfindahl index government (sum of squared seat shares of all parties in the government) [probably a valuable metric]
- HERFTOT - same as above, for all, but very few values. **remove**

--------

- TENLONG - longest tenure of a veto player **remove in favor of strict**
- TENLONG_STRICT - uses TENLONG, restricted to fewer certain leaders 
- TENSHORT - shortest tensure of a veto player **remove in favor of strict**
- TENSHORT_strict - uses TENSHORT, restricted to fewer leaders
- CHECKS - checks and balances
- CHECKS_LAX - **remove**, corresponds to same group as TENLONG (removed), as opposed to CHECKS, which maps to people in TENLONG_STRICT
- STABS, STABS_STRICT - "Stability". use STRICT, as explained in TENLONG.
- STABNS, STABNS_STRICT - similar to STABS, **remove**
- POLARIZ, POLARIZ_STRICT - maximum polarization between the executive party and the four principle parties of the legislature. Use STRING similar to above. 

---

## Manual cleaning

Note that this dataset contained substantial inconsistencies between the way that missing values vs. null values were encoded, and this took a lot of manual study of the data dictionary in order to make the below changes. It is not relevant to waste time explaining each choice. After looking at the data dictionary, the logic behind the choice will no doubt become clear.

In [4]:
# Replace "placeholder" NaN values (as defined by data dictionary) 
# with one consistent np.nan across dataset for consistency

countries.replace(-999, NA_NUMBER, inplace=True)
countries.replace(-999.0, NA_NUMBER, inplace=True)
countries.replace('-999', NA_NUMBER, inplace=True)
countries.replace('-999.0', NA_NUMBER, inplace=True)
countries.replace('NA', NA_STRING, inplace=True)



# One-off cases based on close reading of data dictionary
countries.system.replace(-999, NA_STRING, inplace=True)
countries.system = countries.system.astype('object')
countries.execrlc.replace(0.0, NA_STRING, inplace=True)
countries.execrlc = countries.execrlc.astype('object')
countries.execrel.replace(0.0, OTHER, inplace=True)
countries.execrel.replace(-999.0, NA_STRING, inplace=True)
countries.execrel = countries.execrel.astype('object')
countries.percent1.replace(999.0, NA_NUMBER, inplace=True)
countries.percentl.replace(-99.0, NA_NUMBER, inplace=True)
countries.execrlc.replace(-999.0, NA_STRING, inplace=True)
countries.oppmajs.replace(999.0, NA_NUMBER, inplace=True)
countries.legelec.replace(12.0, NA_NUMBER, inplace=True)
countries.liec.replace(0.0, NA_NUMBER, inplace=True)
countries.eiec.replace(0.0, NA_NUMBER, inplace=True)
countries.housesys.replace('PR', 'Proportional', inplace=True)
countries.housesys.replace(0.5, NA_STRING, inplace=True)
countries.housesys.replace(-999.0, NA_STRING, inplace=True)
countries.housesys = countries.housesys.astype('object')
countries.sensys.replace(-999.0, NA_STRING, inplace=True)
countries.sensys.replace(-888.0, NA_STRING, inplace=True)
countries.sensys.replace(0.5, NA_STRING, inplace=True)
countries.sensys.replace('PR', 'Proportional', inplace=True)
countries.sensys = countries.sensys.astype('object')
countries.thresh.replace(-9999.0, NA_NUMBER, inplace=True)
countries.select.replace(-999.0, NA_STRING, inplace=True)
countries.fraud.replace(0.0, 'OppositionBanned', inplace=True)
countries.fraud.replace(1.0, 'OppositionSuppressed', inplace=True)
countries.fraud.replace(-999.0, NA_STRING, inplace=True)
countries.fraud.fillna(NA_STRING, inplace=True)
countries.muni.replace('Legislature and executive locally elected', 1.0, inplace=True)
countries.muni.replace('Legislature locally elected', 0.5, inplace=True)
countries.muni.replace('No local elections', 0.0, inplace=True)
countries.muni = countries.muni.astype('float')

countries.state.replace('Legislature and executive locally elected', 1.0, inplace=True)
countries.state.replace('Legislature locally elected', 0.5, inplace=True)
countries.state.replace('No local elections', 0.0, inplace=True)
countries.state = countries.state.astype('float')

countries.maj.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.maj.fillna(NA_NUMBER, inplace=True)
countries.maj = countries.maj.astype('float')
countries.partyage.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.partyage.fillna(NA_NUMBER, inplace=True)
countries.partyage = countries.partyage.astype('float')
countries.herfgov.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.herfgov = countries.herfgov.astype('float')
countries.herfopp.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.herfopp = countries.herfopp.astype('float')



countries.frac = countries.frac.astype('object')
countries.frac.fillna(NA_NUMBER, inplace=True)
countries.frac.replace(NA_STRING, NA_NUMBER, inplace=True)

countries.frac = countries.frac.astype('object')
countries.frac.fillna(NA_NUMBER, inplace=True)
countries.frac.replace(NA_STRING, NA_NUMBER, inplace=True)

countries.oppfrac = countries.oppfrac.astype('object')
countries.oppfrac.fillna(NA_NUMBER, inplace=True)
countries.oppfrac.replace(NA_STRING, NA_NUMBER, inplace=True)

countries.govfrac = countries.govfrac.astype('object')
countries.govfrac.fillna(NA_NUMBER, inplace=True)
countries.govfrac.replace(NA_STRING, NA_NUMBER, inplace=True)

countries.tensys_strict.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.tensys_strict = countries.tensys_strict.astype('float')

countries.checks.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.checks = countries.checks.astype('float')

countries.stabs_strict.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.stabs_strict = countries.stabs_strict.astype('float')

countries.tenlong_strict.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.tenlong_strict = countries.tenlong_strict.astype('float')

countries.tenshort_strict.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.tenshort_strict = countries.tenshort_strict.astype('float')

countries.tenshort_strict.replace(NA_STRING, NA_NUMBER, inplace=True)
countries.tenshort_strict = countries.tenshort_strict.astype('float')

countries.polariz.replace(NA_STRING, NA_NUMBER, inplace=True)

In [5]:
# Based on manual review of the data dictionary, explained briefly above
remove_these_features = ['nonchief', 'dateleg', 'dateexec', 'dhondt', 'select', 
                         'stconst', 'gwno', 'numgov', 'numopp', 'herftot', 
                         'tenlong', 'tenshort', 'checks_lax', 'stabns', 
                         'stabns_strict', 'execme', 'tensys', 'stabs', 
                         'execrurl', 'execreg']
countries.drop(remove_these_features, axis=1, inplace=True)

#### Country name codes

Make consistent with other data sources to enable merging. Minor differences in spelling, abbreviations, casing, etc. make this necessary. Process completed in simple Excel file for simplicity. Import that lookup table to convert to new naming schemes.

In [6]:
# Import Excel file containing lookup mapping/dictionary
code_lookup = pd.read_excel('../src/country_codes_governments.xlsx')
name_dict  = dict(zip(code_lookup.combo_original, code_lookup.country_final))
scode_dict = dict(zip(code_lookup.combo_original, code_lookup.scode_final))

# Create ID to be used across all datasets
countries['combo'] = countries.countryname + '_' + countries.ifs
countries['countryname_final'] = countries.combo.map(name_dict)
countries['scode_final'] = countries.combo.map(scode_dict)

# Show results
disp_cols = ['combo', 'countryname', 'countryname_final', 'ifs', 'scode_final']
display(countries[disp_cols].drop_duplicates())

# Drop unused/helper features
countries.drop(['countryname', 'ifs'], axis=1, inplace=True)

,combo,countryname,countryname_final,ifs,scode_final
0,Turk Cyprus_0,Turk Cyprus,IGNORE,0,INGORE
46,Afghanistan_AFG,Afghanistan,Afghanistan,AFG,AFG
92,Angola_AGO,Angola,Angola,AGO,ANG
138,Albania_ALB,Albania,Albania,ALB,ALB
184,UAE_ARE,UAE,United Arab Emirates,ARE,UAE
...,...,...,...,...,...
7970,Yugoslavia_YSR,Yugoslavia,Yugoslavia,YSR,YGS
8016,S. Africa_ZAF,S. Africa,South Africa,ZAF,SAF
8062,Congo (DRC)_ZAR,Congo (DRC),Congo Kinshasa,ZAR,ZAI
8108,Zambia_ZMB,Zambia,Zambia,ZMB,ZAM


In [7]:
# Normalize percentages to [0, 1] instead of [0, 100]
def normalize_to_percent(zero_to_100):
    # -999.0 represents a "null" value in this portion of the analysis
    if_not_999 = lambda x: x/100 if x != -999.0 else x
    percentage = zero_to_100.astype('float').apply(if_not_999)
    return percentage

# Normalize
countries.polariz = countries.polariz.astype('float')
countries.percent1 = normalize_to_percent(countries.percent1)
countries.percentl = normalize_to_percent(countries.percentl)
countries.numvote = normalize_to_percent(countries.numvote)
countries.oppvote = normalize_to_percent(countries.oppvote)

## Feature Engineering

In [8]:
# Combine PERCENT1 and PERCENTL since they cover similar content since 
# PERCENTL is more meaningful where it exists, but it often doesn't exist

last_else_first = lambda first, last: last if last != NA_NUMBER else first

countries['percent1'].fillna(NA_NUMBER, inplace=True)
countries['percentl'].fillna(NA_NUMBER, inplace=True)

countries['percent'] = list(map(last_else_first, 
                                countries['percent1'], 
                                countries['percentl']))

countries['percent'].replace(NA_NUMBER, np.nan, inplace=True)
countries.drop(['percent1', 'percentl'], axis=1, inplace=True)

##### Create ID to be used across all datasets

In [9]:
# "year"_"countrycode"
countries['year_scode'] = countries.year.astype('str')+'_'+countries.scode_final

# Drop unused/helper columns
countries.drop(['year', 'combo'], axis=1, inplace=True)

##### General data cleaning

In [10]:
# FILL IN MISSING VALUES W/CORRECT DATA TYPE
fill_w_object_type = ['system', 'execrlc', 'execrel', 'housesys', 
                      'sensys', 'fraud']

fill_w_num_type = ['yrsoffc', 'finittrm', 'yrcurnt', 'termlimit', 'reelect', 
                   'multpl', 'military', 'defmin', 'prtyin', 'execnat', 
                   'execage', 'allhouse', 'totalseats', 'oppmajh', 'oppmajs', 
                   'legelec', 'exelec', 'liec', 'eiec', 'mdmh', 'mdms', 'ssh', 
                   'pluralty', 'pr', 'thresh', 'cl', 'gq', 'gqi', 'auton', 
                   'muni', 'state', 'author', 'numvote', 'oppvote', 'maj', 
                   'partyage', 'herfgov', 'herfopp', 'frac', 'oppfrac', 
                   'govfrac', 'tensys_strict', 'checks', 'stabs_strict', 
                   'tenlong_strict', 'tenshort_strict', 'polariz', 'percent',
                   'year_scode']

# Fill based on specified data type
for column in fill_w_object_type:
    countries[column].fillna(NA_STRING, inplace=True)
for column in fill_w_num_type:
    countries[column].fillna(NA_NUMBER, inplace=True)


# USE COMMON CATEGORIES OF ALL-ENCOMPASSING FEATURES TO CATEGORIZE MISSING VALS
no_govt_data = countries.loc[(countries.system == NA_STRING) & 
                             (countries.yrcurnt == NA_NUMBER)]
no_govt_data_ind = list(no_govt_data.index)
countries['no_govt_data']= [int(x in no_govt_data_ind) for x in countries.index]

# Drop rows without data
drop_missing = countries.loc[countries['no_govt_data']==1].index
countries.drop(drop_missing, inplace=True)
countries.drop('no_govt_data', axis=1, inplace=True)

# Rename as relevant
countries.rename(columns={'countryname_final': 'country_govt', 
                'scode_final': 'scode_govt'}, inplace=True)

## Export data to SQL 

In [11]:
engine = create_engine('sqlite:///../data/processed/governments.db')

with engine.begin() as connection:
    countries.to_sql(name='governments', 
                     con=connection, 
                     if_exists='replace', 
                     index=False)

##### Final dataset

In [12]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7669 entries, 1 to 8199
Data columns (total 57 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   system           7669 non-null   object 
 1   yrsoffc          7669 non-null   float64
 2   finittrm         7669 non-null   float64
 3   yrcurnt          7669 non-null   float64
 4   termlimit        7669 non-null   float64
 5   reelect          7669 non-null   float64
 6   multpl           7669 non-null   float64
 7   military         7669 non-null   float64
 8   defmin           7669 non-null   float64
 9   prtyin           7669 non-null   float64
 10  execrlc          7669 non-null   object 
 11  execnat          7669 non-null   float64
 12  execrel          7669 non-null   object 
 13  execage          7669 non-null   float64
 14  allhouse         7669 non-null   float64
 15  totalseats       7669 non-null   int16  
 16  oppmajh          7669 non-null   float64
 17  oppmajs       